In [2]:
print( "┏━━━━━━━━━┉┅╍╌" )

if "bot" in locals(): ## cleanup from previous run
    print( "Cleaning up previous run..." )
    if not bot is None:
        try:
            bot.Exit()
        except Exception as error:
            print( "Exception cleaning up previous run -- ", traceback.format_exc() )
        bot = None
    del bot
    gc.collect()
    print( "Cleaned up previous run" )
else:
    print( "Starting in fresh state" )

import GSUbot
import ipywidgets
import traceback
import gc

class NotebookBot():
    def __init__(self):
        self.output = ipywidgets.Output( layout=ipywidgets.Layout( height="20em" ) )
        self.robot = GSUbot.Robot()
        self.controls = self.__buildControls()
    def __del__(self):
        print( "NotebookBot __del__ start" )
        gc.collect()
        print( "NotebookBot __del__ end" )
    def Main(self):
        display( self.controls["display"] )
    def Exit(self):
        with self.output: print( "EXITING" )
        rs = gc.get_referrers( self.robot ); print( "REFERRERS:", len(rs) )
        self.controls["display"].close()
        rs = gc.get_referrers( self.robot ); print( "REFERRERS:", len(rs) )
        self.output = None;   del self.output
        rs = gc.get_referrers( self.robot ); print( "REFERRERS:", len(rs) )
        self.controls = None; del self.controls
        rs = gc.get_referrers( self.robot ); print( "REFERRERS:", len(rs) )
        for r in rs: print( r.keys() )
        self.robot = None;    del self.robot
        gc.collect()
    def print(self, *args):
        with self.output:
            print( *args )
    def ClearOutput(self):
        self.output.clear_output()
        with self.output: print( "CLEARED" )
    def __buildControls(self):
        def make_button( label, on_click, layout=ipywidgets.Layout( width="10em" ), color='#888888' ):
            button = ipywidgets.Button( description=label, layout=layout )
            button.style.button_color = color
            button.on_click(on_click)
            return button

        bNorth = make_button( label="Turn North", on_click=lambda *_:self.robot.spin( heading =   0 ) )
        bWest  = make_button( label="Turn West",  on_click=lambda *_:self.robot.spin( heading = -90 ) )
        bEast  = make_button( label="Turn East",  on_click=lambda *_:self.robot.spin( heading =  90 ) )
        bSouth = make_button( label="Turn South", on_click=lambda *_:self.robot.spin( heading = 180 ) )

        nsew_box = ipywidgets.widgets.VBox( [
            ipywidgets.widgets.HBox( [ bNorth ],      layout=ipywidgets.Layout( justify_content="center" ) ),
            ipywidgets.widgets.HBox( [ bWest, bEast ] ),
            ipywidgets.widgets.HBox( [ bSouth ],      layout=ipywidgets.Layout( justify_content="center" ) ),
        ], layout=ipywidgets.Layout( width="fit-content" ) )

        bForward  = make_button( label="Straight Forward",  on_click=lambda *_: self.robot.forward()           )
        bLeft     = make_button( label="Left Forward",      on_click=lambda *_: self.robot.left()              )
        bStop     = make_button( label="STOP",              on_click=lambda *_: self.robot.stop(), color='red' )
        bRight    = make_button( label="Right Foreward",    on_click=lambda *_: self.robot.right()             )
        bBackward = make_button( label="Straight Backward", on_click=lambda *_: self.robot.backward()          )

        move_box = ipywidgets.widgets.VBox( [
            ipywidgets.widgets.HBox( [ bForward ],            layout=ipywidgets.Layout( justify_content="center" ) ),
            ipywidgets.widgets.HBox( [ bLeft, bStop, bRight ] ),
            ipywidgets.widgets.HBox( [ bBackward ],           layout=ipywidgets.Layout( justify_content="center" ) ),
        ], layout=ipywidgets.Layout( width="fit-content" ) )

        bDistance = make_button( label="Distance", on_click=lambda *_: self.print( "Distance: "+str(self.robot.distance.mm)+"mm" ) )
        bPhoto    = make_button( label="Photo",    on_click=lambda *_: self.print( self.robot.camera.image ) )
        bHeading  = make_button( label="Heading",  on_click=lambda *_: self.print( "Heading: "+str(self.robot.imu.heading)+"°" ) )

        data_box = ipywidgets.widgets.VBox( [ bDistance, bPhoto, bHeading ], layout=ipywidgets.Layout( justify_content="center" ) )
        data_box = ipywidgets.widgets.VBox( [ bDistance, bHeading ], layout=ipywidgets.Layout( justify_content="center" ) )

        bExit = make_button( label="Exit", on_click=lambda *_: self.Exit(), color='red' )

        misc_box = ipywidgets.widgets.VBox( [ bExit ], layout=ipywidgets.Layout( justify_content="center" ) )

        sSpin = ipywidgets.IntSlider( min=-720, max=720, step=1, value=0 )
        tSpin = ipywidgets.IntText()
        lSpin = ipywidgets.jslink( (sSpin,"value"), (tSpin,"value") )  ## this should use .link, but apparently that doesn't exist here

        bClearOutput = make_button( label="Clear Output", on_click=lambda *_: self.ClearOutput() )

        space_width  = ipywidgets.widgets.VBox( layout=ipywidgets.Layout( width="4em" ) )
        space_height = ipywidgets.widgets.VBox( layout=ipywidgets.Layout( height="2em" ) )

        display = ipywidgets.widgets.VBox( [
            ipywidgets.widgets.HBox( [ nsew_box, space_width, data_box, space_width, misc_box ] ),
            space_height,
            ipywidgets.widgets.HBox( [ move_box ] ),
            space_height,
            ipywidgets.widgets.HBox( [ bClearOutput ] ),
            ipywidgets.widgets.HBox( [ self.output ] ),
        ] )

        read = {}

        return { "display": display, "read": read }

bot = NotebookBot()
try:
    bot.Main()
    print( "┣━━━━━━━━━┉┅╍╌" )
    print( "Exiting" )
except Exception as error:
    print( "┣━━━━━━━━━┉┅╍╌" )
    print( "Exception! -- ", traceback.format_exc() )
    bot.Exit()
bot = None
print( "┗━━━━━━━━━┉┅╍╌" )


┏━━━━━━━━━┉┅╍╌
Cleaning up previous run...
Cleaned up previous run
GSUbot.Robot __new__
GSUbot.Robot: reusing singleton instance <Robot@0x7095b750: api_easy=<easygopigo3.EasyGoPiGo3 object at 0x755fa3b0>, hardware=<Hardware@0x6fb06170: api_easy=<easygopigo3.EasyGoPiGo3 object at 0x755fa3b0>, manufacturer=Dexter Industries, model=GoPiGo3, version=3.x.x, serial=04E32C82514E3437324A2020FF040907>, firmware=<Firmware@0x6fb061b0: api_easy=<easygopigo3.EasyGoPiGo3 object at 0x755fa3b0>, version=1.0.0>, vitals=( battery_volts=9.02, regulator_volts=4.975 )>
GSUbot.Robot __init__: already initialized


┣━━━━━━━━━┉┅╍╌
Exiting
┗━━━━━━━━━┉┅╍╌
REFERRERS: 3
REFERRERS: 3
REFERRERS: 3
REFERRERS: 3
dict_keys(['robot'])
dict_keys(['blinkers', 'motor_status', 'reset_all', '_Robot__singleton_instance', 'Camera', 'MotorStatus', 'Hardware', 'stop', '__repr__', 'Distance', '__del__', '__init__', 'IMU', 'Vitals', 'odometer_read', '__doc__', 'backward', '__weakref__', 'left', 'spin', 'odometer_reset', 'Servo', 'eyes', '_Robot__singleton_lock', '__module__', 'Firmware', 'right', 'forward', '__dict__', '__new__'])
dict_keys(['output', 'robot', 'controls'])
